In [1]:
val twitterPath="hdfs:///user/swethakolalapudi/twitter/cache-0-json"

In [2]:
import play.api.libs.json._

In [3]:
val twitterData=sc.textFile(twitterPath).map(x => Json.parse(x))

In [4]:
twitterData.first()\"text"

"Obama vies for health care edge in Florida - http://t.co/OcISvreb http://t.co/FsJ7xgGW #florida"

In [10]:
twitterData.filter(x => (x\"user"\"screen_name").toString.contains("realDonaldTrump")).map(x => x\"text").take(10)

Array("Obama asked a 7 yr old for his birth certificate. He's \"in your face\" because the Republicans dropped the ball. (cont) http://t.co/FufZD79U", "Obama is taunting the Republicans on the birther issue. They should call his bluff &amp; demand the REAL facts. He (cont) http://t.co/NWmVp06e", "\"President Obama is the greatest hoax ever perpetrated on the American people\"\n--Clint Eastwood")

In [7]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.Row

In [8]:
val sqlC= new SQLContext(sc) 

In [9]:
val twitterTable=sqlC.read.json(twitterPath)

In [10]:
twitterTable.registerTempTable("twitterTable")


In [11]:
twitterTable.getClass

class org.apache.spark.sql.DataFrame

In [12]:
twitterTable.printSchema()

root
 |-- contributors: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |  

In [30]:
sqlC.sql("Select text, user.screen_name from twitterTable where user.screen_name='realDonaldTrump' limit 10").collect()

Array([Obama asked a 7 yr old for his birth certificate. He's "in your face" because the Republicans dropped the ball. (cont) http://t.co/FufZD79U,realDonaldTrump], [Obama is taunting the Republicans on the birther issue. They should call his bluff &amp; demand the REAL facts. He (cont) http://t.co/NWmVp06e,realDonaldTrump], ["President Obama is the greatest hoax ever perpetrated on the American people"
--Clint Eastwood,realDonaldTrump])

In [17]:
val trumpTweets = sqlC.sql("Select text, user.screen_name,entities from twitterTable where user.screen_name='realDonaldTrump' limit 10")

In [18]:
trumpTweets.select("text").take(10)

Array([Obama asked a 7 yr old for his birth certificate. He's "in your face" because the Republicans dropped the ball. (cont) http://t.co/FufZD79U], [Obama is taunting the Republicans on the birther issue. They should call his bluff &amp; demand the REAL facts. He (cont) http://t.co/NWmVp06e], ["President Obama is the greatest hoax ever perpetrated on the American people"
--Clint Eastwood])

In [43]:
val hashtags = sqlC.sql("select entities.hashtags.text from twitterTable where entities.hashtags is not null")